In [5]:
import pandas as pd
import numpy as np

DF = pd.read_csv('WiFi/wifigarden.csv') 
DF1 =  pd.read_csv('Lora/loracalib.csv') 

B1 = DF['SSID']=='FTM_4'
DF = DF[B1]

print("WiFi data")
print("AVG RSSI at 1 meter", DF['RSSI'].mean())
print("VARIANCE RSSI at 1 meter", DF['RSSI'].var())
print("MAX RSSI at 1 meter", DF['RSSI'].max())
print("MIN RSSI at 1 meter", DF['RSSI'].min())

print("AVG EST_DIST at 1 meter", DF['DIST'].mean()/100)
print("VARIANCE EST_DIST at 1 meter", DF['DIST'].var()/100)
print("MAX EST_DIST at 1 meter", DF['DIST'].max()/100)
print("MIN EST_DIST at 1 meter", DF['DIST'].min()/100)

calib = DF['DIST'].mean()/100 - 1
print("Suggested calibration: ", round(calib,2))
Calib_offset = 52.46368015 #175ns covered distance
print('****************************************')
print("LoRa data")
print("AVG RSSI at 1 meter", DF1['RSSI'].mean())
print("VARIANCE RSSI at 1 meter", DF1['RSSI'].var())
print("MAX RSSI at 1 meter", DF1['RSSI'].max())
print("MIN RSSI at 1 meter", DF1['RSSI'].min())

print("AVG EST_DIST at 1 meter", DF1['DIST'].mean())
print("VARIANCE EST_DIST at 1 meter", DF1['DIST'].var())
print("MAX EST_DIST at 1 meter", DF1['DIST'].max())
print("MIN EST_DIST at 1 meter", DF1['DIST'].min())

calib = DF1['DIST'].mean() - 1
print("Suggested calibration: ", round(calib,2))
Calib_offset = 52.46368015 #175ns covered distance

WiFi data
AVG RSSI at 1 meter -26.317596566523605
VARIANCE RSSI at 1 meter 0.2946144261387234
MAX RSSI at 1 meter -25.0
MIN RSSI at 1 meter -27.0
AVG EST_DIST at 1 meter 54.307939914163086
VARIANCE EST_DIST at 1 meter 7.952822003784209
MAX EST_DIST at 1 meter 55.0
MIN EST_DIST at 1 meter 52.8
Suggested calibration:  53.31
****************************************
LoRa data
AVG RSSI at 1 meter -60.054794520547944
VARIANCE RSSI at 1 meter 40.60806697108066
MAX RSSI at 1 meter -55
MIN RSSI at 1 meter -112
AVG EST_DIST at 1 meter 58.804060482054794
VARIANCE EST_DIST at 1 meter 33200.75286959633
MAX EST_DIST at 1 meter 1454.5448
MIN EST_DIST at 1 meter 10.07361794
Suggested calibration:  57.8


In [61]:
def estimate_distance(power_received, params=None):
    """This function returns an estimated distance range
       given a single radio signal strength (RSS) reading
       (received power measurement) in dBm.
    Parameters:
        power_received (float): RSS reading in dBm
        params (4-tuple float): (d_ref, power_ref, path_loss_exp, stdev_power)
            d_ref is the reference distance in m
            power_ref is the received power at the reference distance
            path_loss_exp is the path loss exponent
            stdev_power is standard deviation of received Power in dB
    Returns:
        (d_est, d_min, d_max): a 3-tuple of float values containing
            the estimated distance, as well as the minimum and maximum
            distance estimates corresponding to the uncertainty in RSS,
            respectively, in meters rounded to two decimal points
    """

    if params is None:
        params = (1.0, -32.1, 4, np.sqrt(0.4))
          # the above values are arbitrarily chosen "default values"
          # should be changed based on measurements

    d_ref = params[0] # reference distance
    power_ref = params[1] # mean received power at reference distance
    path_loss_exp = params[2] # path loss exponent
    stdev_power = params[3] # standard deviation of received power

    uncertainty = 2*stdev_power # uncertainty in RSS corresponding to 95.45% confidence

    d_est = d_ref*(10**(-(power_received - power_ref)/(10*path_loss_exp)))
    d_min = d_ref*(10**(-(power_received - power_ref + uncertainty)/(10*path_loss_exp)))
    d_max = d_ref*(10**(-(power_received - power_ref - uncertainty)/(10*path_loss_exp)))

    return (np.round(d_est,2), np.round(d_min,2), np.round(d_max,2))

In [67]:
print(estimate_distance(-40))
print(estimate_distance(-39))
print(estimate_distance(-41))

(1.58, 1.47, 1.69)
(1.49, 1.38, 1.6)
(1.67, 1.55, 1.8)


In [51]:
estimate_distance(-46)

import localization as lx

#Trilateration
P=lx.Project(mode='3D',solver='LSE_')
#Anchors
P.add_anchor('FTM_1',(-21,30,0))
P.add_anchor('FTM_2',(-212,30,0))
P.add_anchor('FTM_3',(-19,222,0))
#Gateway
t,label=P.add_target()



In [44]:
t.add_measure('FTM_1',5560-5336)
t.add_measure('FTM_2',5630-5336)
t.add_measure('FTM_3',5730-5336)

P.solve()

# Then the target location is:

print(t.loc)

LSE Geolocating...
p(-8.040012609157138,-182.5190088486153,0.0)


In [68]:
#Trilateration
P=lx.Project(mode='3D',solver='LSE')
#Anchors
P.add_anchor('FTM_1',(-21,30,0))
P.add_anchor('FTM_2',(-212,30,0))
P.add_anchor('FTM_3',(-19,222,0))
#Gateway
t,label=P.add_target()

t.add_measure('FTM_1',estimate_distance(-40)[0]*100)
t.add_measure('FTM_2',estimate_distance(-39)[0]*100)
t.add_measure('FTM_3',estimate_distance(-41)[0]*100)

P.solve()

# Then the target location is:

print(t.loc)

LSE Geolocating...
p(-144.03753513236472,141.81000531813754,0.0)
